In [2]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

%matplotlib inline
plt.style.use('seaborn')

In [3]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

print("Версия Hyperopt : {}".format(hyperopt.__version__))

Версия Hyperopt : 0.2.7


In [81]:
import optuna

print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 3.1.1


In [4]:
data = pd.read_csv('data/_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X = data.drop('Activity', axis=1)
y = data.Activity
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

# Линейная модель

In [36]:
regression = linear_model.LogisticRegression(max_iter=1000, random_state=42)
regression.fit(X_train, y_train)
y_test_pred = regression.predict(X_test)
print('метрика f1 на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))

метрика f1 на тестовом наборе: 0.7773766546329723


### GridSearcvCV

In [37]:
param_grid = {'penalty': ['l2', 'none'] ,#тип регурялизации
              'solver': ['lbfgs', 'saga'], #алгоритм оптимизации
              }

In [19]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(
    estimator=regression, 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

f1_score на тестовом наборе: 0.7773766546329723
Наилучшие значения гиперпараметров: {'penalty': 'l2', 'solver': 'lbfgs'}


### RandomizedSearchCV

In [24]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=regression,
    param_distributions=param_grid,
    cv=5,
    n_iter=10,
    n_jobs=-1
)
random_search.fit(X_train,y_train)
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


f1_score на тестовом наборе: 0.7773766546329723
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2'}


### HyperOpt

In [77]:
param_grid = {'penalty':hp.choice(label='penalty', options=['l2', 'none']) ,#тип регурялизации
              'solver':hp.choice(label='solver', options=['lbfgs', 'saga']), #алгоритм оптимизации
              }

In [79]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {
        'penalty': params['penalty'],
        'solver': params['solver']
    }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state)

    # обучаем модель
    #model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [80]:
# начинаем подбор гиперпараметров

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=param_grid, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [01:39<00:00,  4.96s/trial, best loss: -0.7793179449996321]
Наилучшие значения гиперпараметров {'penalty': 0, 'solver': 1}


### Optuna

In [89]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', choices=['l2', 'none'])
  solver = trial.suggest_categorical('solver', choices=['lbfgs', 'saga'])

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          random_state=random_state)
  # обучаем модель
  #model.fit(X_train, y_train)
  #score = metrics.f1_score(y_train, model.predict(X_train))

  # Без кросс-валидации модель переобучается
  score = cross_val_score(model, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()
  return score

In [90]:
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2023-04-17 12:58:07,814] A new study created in memory with name: LogisticRegression
[I 2023-04-17 12:58:16,976] Trial 0 finished with value: 0.7784186232901155 and parameters: {'penalty': 'none', 'solver': 'saga'}. Best is trial 0 with value: 0.7784186232901155.
[I 2023-04-17 12:58:24,651] Trial 1 finished with value: 0.7784186232901155 and parameters: {'penalty': 'none', 'solver': 'saga'}. Best is trial 0 with value: 0.7784186232901155.
[I 2023-04-17 12:58:26,526] Trial 2 finished with value: 0.7507880100918504 and parameters: {'penalty': 'none', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.7784186232901155.
[I 2023-04-17 12:58:28,502] Trial 3 finished with value: 0.7507880100918504 and parameters: {'penalty': 'none', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.7784186232901155.
[I 2023-04-17 12:58:36,241] Trial 4 finished with value: 0.7793179449996321 and parameters: {'penalty': 'l2', 'solver': 'saga'}. Best is trial 4 with value: 0.7793179449996321.
[I 2023-04-17 12:

In [91]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {}".format(study.best_value))
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(random_state=random_state, 
                                        penalty=study.best_params['penalty'],
                                        solver=study.best_params['solver'])
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'saga'}
f1_score на обучающем наборе: 0.7793179449996321
f1_score на тестовом наборе: 0.7753623188405797


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


# Случайный лес

In [15]:
forest = ensemble.RandomForestClassifier(random_state=42)
forest.fit(X_train, y_train)
y_test_pred = forest.predict(X_test)
print('метрика f1 на тестовом наборе: {}'.format(metrics.f1_score(y_test,y_test_pred)))
print('accuracy на тестовом наборе: {}'.format(metrics.accuracy_score(y_test, y_test_pred)))

метрика f1 на тестовом наборе: 0.8048484848484848
accuracy на тестовом наборе: 0.7856191744340879


### GridSearchCV

In [16]:
param_grid = {'n_estimators': list(range(80, 150, 10)),
              'min_samples_leaf': [1,2,3,4,5]}

In [17]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(
    estimator=forest, 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))
print('accuracy на тестовом наборе: {}'.format(metrics.accuracy_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

f1_score на тестовом наборе: 0.8014616321559075
accuracy на тестовом наборе: 0.7829560585885486
Наилучшие значения гиперпараметров: {'min_samples_leaf': 2, 'n_estimators': 100}


### RandomizedSearchCV

In [18]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=forest,
    param_distributions=param_grid,
    cv=5,
    n_iter=10,
    n_jobs=-1
)
random_search.fit(X_train,y_train)
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

f1_score на тестовом наборе: 0.8009708737864076
Наилучшие значения гиперпараметров: {'n_estimators': 120, 'min_samples_leaf': 2}


### HyperOpt

In [29]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 90, 180, 10),
        'max_depth' : hp.quniform('max_depth', 15, 40, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 5, 1)
      }

In [45]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
               'max_depth' : int(params['max_depth']),
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    #model.fit(X, y)
    #score = metrics.f1_score(y, model.predict(X))
    
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    
    # Без кросс-валидации модель переобучается
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [46]:
# начинаем подбор гиперпараметров

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [01:07<00:00,  3.39s/trial, best loss: -0.8181415303558588]
Наилучшие значения гиперпараметров {'max_depth': 30.0, 'min_samples_leaf': 2.0, 'n_estimators': 130.0}


In [47]:
model = ensemble.RandomForestClassifier(random_state=random_state, 
                                        n_estimators=int(best['n_estimators']),  
                                        min_samples_leaf=int(best['min_samples_leaf']))
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))
print('accuracy на тестовом наборе: {}'.format(metrics.accuracy_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.7965895249695493
accuracy на тестовом наборе: 0.7776298268974701


### Optuna

In [85]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 90, 180, 10)
  max_depth = trial.suggest_int('max_depth', 15, 40, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  #model.fit(X_train, y_train)
  #score = metrics.f1_score(y_train, model.predict(X_train))

  # Без кросс-валидации модель переобучается
  score = cross_val_score(model, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()
  return score
  

In [86]:
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2023-04-17 12:50:33,614] A new study created in memory with name: RandomForestClassifier
[I 2023-04-17 12:50:37,975] Trial 0 finished with value: 0.8095360556355915 and parameters: {'n_estimators': 120, 'max_depth': 35, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8095360556355915.
[I 2023-04-17 12:50:42,141] Trial 1 finished with value: 0.811651038841512 and parameters: {'n_estimators': 180, 'max_depth': 24, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.811651038841512.
[I 2023-04-17 12:50:46,187] Trial 2 finished with value: 0.8116813946107635 and parameters: {'n_estimators': 180, 'max_depth': 29, 'min_samples_leaf': 4}. Best is trial 2 with value: 0.8116813946107635.
[I 2023-04-17 12:50:49,979] Trial 3 finished with value: 0.8140958029878858 and parameters: {'n_estimators': 140, 'max_depth': 36, 'min_samples_leaf': 1}. Best is trial 3 with value: 0.8140958029878858.
[I 2023-04-17 12:50:53,046] Trial 4 finished with value: 0.8096209605581792 and parameters: {'n_es

In [88]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {}".format(study.best_value))
best=study.best_params
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(random_state=random_state, 
                                        n_estimators=int(best['n_estimators']),  
                                        max_depth=int(best['max_depth']),
                                        min_samples_leaf=int(best['min_samples_leaf']))
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'n_estimators': 150, 'max_depth': 16, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.8185276610301819
f1_score на тестовом наборе: 0.8038740920096853
